In [55]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys, io, re, string

max length:  60


In [67]:
shakes_lines = []
poem_starts = []
next_ln = False

with open("data/shakespeare.txt") as f:
    
    maxlen = max([len(ln) for ln in text.split('\n')]) + 1
    print('max length: ', maxlen)
    
    # Read in all lines
    lines = f.readlines()
    for line in lines[1:]:
        
        # replace poem breaks with ~
        if re.match('\s+\d+', line):
            shakes_lines.append('~')
            next_ln = True
            continue
            
        # get rid of blank lines
        seq = line.strip()
        if len(seq) < 3:
            continue
        # remove punctuation
        seq = seq.translate(str.maketrans('', '', string.punctuation))
        # make lowercase
        seq = seq.lower()
        #print(seq)
        shakes_lines.append(seq)
        
        if next_ln:
            poem_starts.append(seq)
    
processed_text = '\n'.join(shakes_lines)
print(processed_text[:60*20])
print(poem_starts[:5])

max length:  60
from fairest creatures we desire increase
that thereby beautys rose might never die
but as the riper should by time decease
his tender heir might bear his memory
but thou contracted to thine own bright eyes
feedst thy lights flame with selfsubstantial fuel
making a famine where abundance lies
thy self thy foe to thy sweet self too cruel
thou that art now the worlds fresh ornament
and only herald to the gaudy spring
within thine own bud buriest thy content
and tender churl makst waste in niggarding
pity the world or else this glutton be
to eat the worlds due by the grave and thee
~
when forty winters shall besiege thy brow
and dig deep trenches in thy beautys field
thy youths proud livery so gazed on now
will be a tattered weed of small worth held
then being asked where all thy beauty lies
where all the treasure of thy lusty days
to say within thine own deep sunken eyes
were an alleating shame and thriftless praise
how much more praise deserved thy beautys use
if thou co

In [68]:
chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
print(chars)
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

total chars: 29
['\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~']


In [69]:
# cut the text in semi-redundant sequences of maxlen characters
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

print(x.shape, y.shape)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        try:
            x[i, t, char_index[char]] = 1
        except:
            print(i, t, char_index[char])
    y[i, char_index[next_chars[i]]] = 1

nb sequences: 30261
Vectorization...
(30261, 60, 29) (30261, 29)


In [70]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(clipnorm=1)
model.compile(optimizer, 'categorical_crossentropy', 
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

In [71]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [72]:
def on_epoch_end(epoch, _, epochs_split=5):
    # Function invoked at end of each epoch. Prints generated text.
    if epoch % epochs_split == 0:
        print()
        print(f'-- generating text after Epoch {epoch} --')

        start_index = random.randint(0, len(processed_text) - maxlen - 1)
        for temp in [0.2, 0.5, 1.0]:
            print(f'-- Temperature: {temp} --')

            generated = ''
            sentence = processed_text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            lines = 1
            while lines < 10
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_index[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, temp)
                next_char = index_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
30261/30261 [==============================] - 6s 210us/step - loss: 2.7940 - categorical_accuracy: 0.2028 - categorical_crossentropy: 2.7940

-- generating text after Epoch 0 --
-- Temperature: 0.2 --
----- Generating with seed: "ection should he live
and with his presence grace impiety
th"
ection should he live
and with his presence grace impiety
th th th s th th the the th se th s th th s se me th s th s se th an se s th s th s th th s th s s th s the s th th mh se the se th th se th th s th s th th th the s the sat the th s th s th th th s ae th th se th th s mhe se th se th th se s th th th th th s th s th th th the se sh th se s se th re so s th s nt the sh th th th th th me s th th  he the s se se s th s th s the  he th the th th s the 
-- Temperature: 0.5 --
----- Generating with seed: "ection should he live
and with his presence grace impiety
th"
ection should he live
and with his presence grace impiety
the swf the sh ns dr
i se  sos lo tes ss n me th  hh th to th c